In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite", echo=False)

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
engine.execute('SELECT * FROM measurement LIMIT 5').fetchall()

In [ ]:
engine.execute('SELECT * FROM station LIMIT 5').fetchall()

In [ ]:
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
inspector = inspect(engine)
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
last_month_data = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
last_month_data

In [ ]:
# Calculate the date 1 year ago from the last data point in the database
date = dt.date(2017, 8, 23) - dt.timedelta(days=365)

In [ ]:
# Perform a query to retrieve the data and precipitation scores
last_month = dt.datetime(2017, 8, 23)
first_month = dt.datetime(2016, 8, 23)
last_twelve_months = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date > first_month).filter(Measurement.date < last_month).all()

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
last_twelve_dates = [l[0] for l in last_twelve_months]
last_twelve_precipation = [l[1] for l in last_twelve_months]
all_months_df = pd.DataFrame({'Date':last_twelve_dates,'Precipitation':last_twelve_precipation})
all_months_df.set_index('Date',inplace=True)
all_months_df.head()

In [ ]:
# Sort the dataframe by date
all_months_df = all_months_df.sort_values(by='Date')
all_months_df.head()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
fig, ax = plt.subplots(figsize=(13,7))
all_months_df.plot(ax=ax)
ax.set_ylabel("Precipitation")
ax.set_xlabel("Date")
plt.savefig("precipitation_last_12_months.png")
plt.show()

![precipitation](Images/precipitation.png)

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
all_months_df.describe()

![describe](Images/describe.png)

In [ ]:
# Design a query to show how many stations are available in this dataset?
number_of_stations = session.query(Measurement).group_by(Measurement.station).count()
number_of_stations

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
most_active = session.query(Measurement.station, Station.name, func.count(Measurement.id)).\
    filter(Measurement.station == Station.station).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.id).desc()).statement
most_active = pd.read_sql_query(most_active, session.bind)
most_active

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?

temps = [Measurement.id,
    func.min(Measurement.tobs), 
       func.max(Measurement.tobs), 
       func.avg(Measurement.tobs)]
station_temps = session.query(*temps).\
    filter(Measurement.date > first_month).filter(Measurement.date < last_month).\
    filter(Measurement.station == Station.station).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.station).desc()).all()
station_temps

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temps_df = pd.DataFrame(station_temps, columns=['ID','MIN', 'MAX', 'AVG'])
temps_df.set_index('ID', inplace=True)
temps_df.plot.hist()
plt.tight_layout()
plt.show()

![precipitation](Images/station-histogram.png)

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# print(calc_temps('2016-08-23', '2017-08-23'))
trip_temps = calc_temps('2016-08-23', '2017-08-23')
trip_temps

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
fig = plt.figure()
ax.set_title('Trip Avg Temp')
x = [...]
y = ['Average Temp']
ax.bar(x,y)
plt.show()

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

